In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import time
import pandas as pd

In [ ]:
print("Carregando dados")
data = np.loadtxt('./data/wine.data', delimiter=',')

X, y = data[:, 1:], data[:,0]

# Transforma em problema de classificação binária
idxs = [i for i in range(len(y)) if y[i] == 1 or y[i] == 2]
X, y = X[idxs], y[idxs]

# Normaliza os dados
X = (X - X.mean(axis=0))/(X.max(axis=0) - X.min(axis=0))
X = np.hstack((X, np.ones(len(X)).reshape(len(X),1)))

# Transforma variável target
y = np.array(list(map(lambda x: 0 if x == 1 else 1, y)))

print(f"Dataset: {X.shape[0]} amostras, {X.shape[1]} features")

In [ ]:
print("\nTreinando modelo benchmark (Scikit-Learn)")
reg = LogisticRegression(solver='sag', C=100000, max_iter=10000).fit(X, y)
L_star = log_loss(y, reg.predict_proba(X))
print(f"Loss L* (benchmark) = {L_star:.10f}")

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cross_entropy_loss(y, y_pred):
    epsilon = 1e-15  # Para evitar log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -(1/len(y))*np.sum(y*np.log(y_pred) + (1 - y)*np.log(1 - y_pred))
    return loss

def cross_entropy_grad(y, y_pred, X):
    return list(np.dot((y_pred - y), X)[0])

def compute_loss_for_weights(w, X, y):
    """Calcula loss para um dado conjunto de pesos"""
    y_pred = sigmoid(np.dot(w.T, X.T))
    return cross_entropy_loss(y, y_pred)

In [ ]:
print("\n" + "="*70)
print("DESCIDA COORDENADA COM GRADIENTE (14 PESOS)")
print("="*70)

w_gradient = np.zeros(14).reshape(14, 1)
eta = 0.01
num_features = 14

# Parâmetros de convergência
patience = 1000  # Número de iterações sem melhoria significativa
min_improvement = 1e-6  # Melhoria mínima considerada significativa
max_iter = 1000000  # Limite máximo de iterações

loss_gradient = []
weights_history_gradient = []
gradient_selections = np.zeros(num_features)
iteration_log = []  # Para tabela de evolução

best_loss = float('inf')
no_improvement_count = 0

print(f"\nParâmetros de convergência:")
print(f"  - Paciência: {patience} iterações")
print(f"  - Melhoria mínima: {min_improvement}")
print(f"  - Máximo de iterações: {max_iter}")

start_time = time.time()
pbar = tqdm(desc="Descida com Gradiente (14 pesos)", total=max_iter)

for t in range(max_iter):
    y_pred = sigmoid(np.dot(w_gradient.T, X.T))
    current_loss = cross_entropy_loss(y, y_pred)
    loss_gradient.append(current_loss)
    
    grad = cross_entropy_grad(y, y_pred, X)
    
    # Seleciona a coordenada com maior gradiente (em valor absoluto)
    grad_abs = [abs(g) for g in grad]
    best_index = grad_abs.index(max(grad_abs))
    gradient_selections[best_index] += 1
    
    # Atualiza apenas o peso com maior gradiente
    w_gradient[best_index] = w_gradient[best_index] - eta*grad[best_index]
    
    # Salva histórico
    weights_history_gradient.append(w_gradient.flatten().copy())
    
    # Log a cada 1000 iterações
    if t % 1000 == 0 or t < 10:
        iteration_log.append({
            'Iteração': t,
            'Loss': current_loss,
            'Melhor Índice': best_index,
            'Gradiente Max': max(grad_abs),
            'Delta Loss': current_loss - best_loss if t > 0 else 0
        })
    
    # Verifica convergência
    if current_loss < best_loss - min_improvement:
        best_loss = current_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1
    
    # Critério de parada
    if no_improvement_count >= patience:
        print(f"\nConvergência atingida na iteração {t}")
        print(f"  Sem melhoria significativa por {patience} iterações")
        pbar.close()
        break
    
    pbar.update(1)
    if t == max_iter - 1:
        print(f"\nMáximo de iterações ({max_iter}) atingido")
        pbar.close()

time_gradient = time.time() - start_time
weights_history_gradient = np.array(weights_history_gradient)
total_iterations_14 = len(loss_gradient)

y_pred_gradient = sigmoid(np.dot(w_gradient.T, X.T))
y_pred_gradient_class = np.array(list(map(lambda x: 1 if x >= 0.5 else 0, y_pred_gradient.flatten())))

print(f"\nResultados:")
print(f"  Tempo de execução: {time_gradient:.2f}s")
print(f"  Total de iterações: {total_iterations_14:,}")
print(f"  Acurácia: {accuracy_score(y, y_pred_gradient_class):.4f}")
print(f"  Loss final: {loss_gradient[-1]:.10f}")

print("\n" + "="*70)
print("TABELA DE EVOLUÇÃO (Primeiras e últimas iterações)")
print("="*70)
df_log = pd.DataFrame(iteration_log)
print("\nPrimeiras 5 iterações:")
print(df_log.head(5).to_string(index=False))
print("\nÚltimas iterações registradas:")
print(df_log.tail(5).to_string(index=False))

CRITÉRIOS DE SELEÇÃO DOS 2 PESOS MAIS IMPORTANTES

Frequência de Seleção: Quantas vezes o peso foi escolhido para atualização -> Indica quão frequentemente o peso precisa ser ajustado

Magnitude Final: Valor absoluto do peso após convergência -> Indica o impacto direto do peso na predição

Variância: Quanto o peso mudou durante o treinamento -> Indica a sensibilidade e importância do peso no processo de otimização

In [ ]:
print("\n" + "="*70)
print("ANÁLISE DE IMPORTÂNCIA DOS PESOS")
print("="*70)

# Critério 1: Frequência de seleção durante otimização
freq_normalized = gradient_selections / gradient_selections.sum()

# Critério 2: Magnitude do peso final
weight_magnitude = np.abs(w_gradient.flatten())
magnitude_normalized = weight_magnitude / weight_magnitude.sum()

# Critério 3: Variância do peso durante treinamento
weight_variance = np.var(weights_history_gradient, axis=0)
variance_normalized = weight_variance / weight_variance.sum()

# Score combinado (média dos 3 critérios)
importance_score = (freq_normalized + magnitude_normalized + variance_normalized) / 3

# Seleciona os 2 pesos mais importantes
top_2_indices = np.argsort(importance_score)[-2:][::-1]

print(f"\nTOP 2 PESOS MAIS IMPORTANTES:")
for rank, idx in enumerate(top_2_indices, 1):
    print(f"\n#{rank} - Peso w_{idx}:")
    print(f"  - Frequência de seleção: {gradient_selections[idx]:.0f} vezes ({freq_normalized[idx]*100:.2f}%)")
    print(f"  - Magnitude final: {weight_magnitude[idx]:.6f} ({magnitude_normalized[idx]*100:.2f}%)")
    print(f"  - Variância: {weight_variance[idx]:.6f} ({variance_normalized[idx]*100:.2f}%)")
    print(f"  - Score de importância: {importance_score[idx]:.6f}")

In [ ]:
print("\n" + "="*70)
print("RETREINAMENTO COM APENAS OS 2 PESOS MAIS IMPORTANTES")
print("="*70)

# Cria dataset reduzido com as 2 features mais importantes + BIAS
# O bias (coluna de 1s) é sempre a última coluna (índice 13)
if 13 in top_2_indices:
    # Se o bias já está nos top 2, usa apenas eles
    X_reduced = X[:, top_2_indices]
    print(f"Dataset reduzido: features {top_2_indices[0]} e {top_2_indices[1]} (inclui bias)")
else:
    # Se o bias não está nos top 2, adiciona ele manualmente
    feature_indices = list(top_2_indices) + [13]
    X_reduced = X[:, feature_indices]
    print(f"Dataset reduzido: features {top_2_indices[0]}, {top_2_indices[1]} + bias (w_13)")
    
num_weights_2d = X_reduced.shape[1]

w_2d = np.zeros(num_weights_2d).reshape(num_weights_2d, 1)
loss_2d = []
weights_history_2d = []
iteration_log_2d = []

best_loss_2d = float('inf')
no_improvement_count_2d = 0

start_time = time.time()
pbar = tqdm(desc=f"Descida com Gradiente ({num_weights_2d} pesos)", total=max_iter)

for t in range(max_iter):
    y_pred = sigmoid(np.dot(w_2d.T, X_reduced.T))
    current_loss = cross_entropy_loss(y, y_pred)
    loss_2d.append(current_loss)
    
    grad = list(np.dot((y_pred - y), X_reduced)[0])
    grad_abs = [abs(g) for g in grad]
    best_index = grad_abs.index(max(grad_abs))
    
    w_2d[best_index] = w_2d[best_index] - eta*grad[best_index]
    weights_history_2d.append(w_2d.flatten().copy())
    
    # Log a cada 100 iterações (mais frequente para ver melhor)
    if t % 100 == 0 or t < 10:
        log_entry = {
            'Iteracao': t,
            'Loss': current_loss,
            'Coord. Atualizada': f'w_{feature_indices[best_index] if num_weights_2d == 3 else top_2_indices[best_index]}',
            'Delta Loss': current_loss - best_loss_2d if t > 0 else 0
        }
        # Adiciona os valores dos pesos
        for i in range(num_weights_2d):
            weight_name = f'w_{feature_indices[i]}' if num_weights_2d == 3 else f'w_{top_2_indices[i]}'
            log_entry[weight_name] = w_2d[i, 0]
        iteration_log_2d.append(log_entry)
    
    # Verifica convergência
    if current_loss < best_loss_2d - min_improvement:
        best_loss_2d = current_loss
        no_improvement_count_2d = 0
    else:
        no_improvement_count_2d += 1
    
    # Critério de parada
    if no_improvement_count_2d >= patience:
        print(f"\nConvergência atingida na iteração {t}")
        print(f"  Sem melhoria significativa por {patience} iterações")
        pbar.close()
        break
    
    pbar.update(1)
    if t == max_iter - 1:
        print(f"\nMáximo de iterações ({max_iter}) atingido")
        pbar.close()

time_2d = time.time() - start_time
weights_history_2d = np.array(weights_history_2d)
total_iterations_2 = len(loss_2d)

y_pred_2d = sigmoid(np.dot(w_2d.T, X_reduced.T))
y_pred_2d_class = np.array(list(map(lambda x: 1 if x >= 0.5 else 0, y_pred_2d.flatten())))

print(f"\nResultados:")
print(f"  Tempo de execucao: {time_2d:.2f}s")
print(f"  Total de iteracoes: {total_iterations_2:,}")
print(f"  Acuracia: {accuracy_score(y, y_pred_2d_class):.4f}")
print(f"  Loss final: {loss_2d[-1]:.10f}")
print(f"  Pesos utilizados: {num_weights_2d} ({top_2_indices[0]}, {top_2_indices[1]}" + 
      (f", 13 (bias))" if num_weights_2d == 3 else ")"))

# Mostra tabela de evolução 2D
print("\n" + "="*70)
print(f"TABELA DE EVOLUCAO - {num_weights_2d} PESOS (Amostragem)")
print("="*70)
df_log_2d = pd.DataFrame(iteration_log_2d)
print("\nPrimeiras 5 iteracoes:")
print(df_log_2d.head(5).to_string(index=False))
print("\nUltimas iteracoes registradas:")
print(df_log_2d.tail(5).to_string(index=False))

In [ ]:
print("\n" + "="*70)
print("COMPARAÇÃO FINAL DE RESULTADOS")
print("="*70)

print("\nCOMPARACAO DE PERDA (LOSS):")
print(f"  Benchmark (Scikit-Learn):         {L_star:.10f}")
print(f"  Descida com Gradiente (14 pesos): {loss_gradient[-1]:.10f}  (diff: +{loss_gradient[-1] - L_star:.10f})")
print(f"  Descida com Gradiente ({num_weights_2d} pesos):  {loss_2d[-1]:.10f}  (diff: +{loss_2d[-1] - L_star:.10f})")

print("\nCOMPARACAO DE TEMPO:")
print(f"  Descida com Gradiente (14 pesos): {time_gradient:.2f}s  ({total_iterations_14:,} iteracoes)")
print(f"  Descida com Gradiente ({num_weights_2d} pesos):  {time_2d:.2f}s  ({total_iterations_2:,} iteracoes)")
print(f"  Speedup: {time_gradient/time_2d:.2f}x mais rapido")

print("\nEFICIENCIA (Loss vs Tempo):")
print(f"  14 pesos: {loss_gradient[-1]:.6f} loss em {time_gradient:.2f}s")
print(f"  {num_weights_2d} pesos:  {loss_2d[-1]:.6f} loss em {time_2d:.2f}s")
degradation = ((loss_2d[-1] - loss_gradient[-1])/loss_gradient[-1]*100)
speedup_pct = ((time_gradient - time_2d)/time_gradient*100)
print(f"  Trade-off: {degradation:.2f}% mais loss para {speedup_pct:.2f}% menos tempo")

In [ ]:
# Comparação de Loss ao longo das iterações
plt.figure(figsize=(14, 6))
plt.plot(loss_gradient, 'r-', label='Descida com Gradiente (14 pesos)', alpha=0.7, linewidth=1.5)
plt.plot(loss_2d, 'g-', label=f'Descida com Gradiente ({num_weights_2d} pesos)', alpha=0.7, linewidth=1.5)
plt.axhline(y=L_star, color='orange', linestyle='--', linewidth=2, label='L* (Benchmark)')
plt.title('Comparação: Iteração vs Loss', fontsize=18, fontweight='bold')
plt.xlabel('Iteração', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.xlim(0, max(total_iterations_14, total_iterations_2))
plt.ylim(-0.01, 0.15)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Evolução dos 14 pesos (Descida com Gradiente)
fig, axes = plt.subplots(4, 4, figsize=(20, 16))
fig.suptitle('Evolução de Cada Peso - Descida Coordenada com Gradiente (14 pesos)', 
             fontsize=24, fontweight='bold')
axes_flat = axes.flatten()

for i in range(num_features):
    ax = axes_flat[i]
    color = 'red' if i in top_2_indices else 'steelblue'
    linewidth = 2.5 if i in top_2_indices else 1.0
    ax.plot(weights_history_gradient[:, i], color=color, linewidth=linewidth)
    
    title = f'Peso w_{i}'
    if i in top_2_indices:
        title += ' (TOP 2)'
    ax.set_title(title, fontsize=12, fontweight='bold' if i in top_2_indices else 'normal')
    ax.set_xlabel('Iteração', fontsize=10)
    ax.set_ylabel('Valor', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.6)

axes_flat[14].axis('off')
axes_flat[15].axis('off')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Criar grade de loss para visualizar espaço de soluções
print("\nCalculando espaco de solucoes")

# Encontra os limites da trajetória com margem
w0_min, w0_max = weights_history_2d[:, 0].min(), weights_history_2d[:, 0].max()
w1_min, w1_max = weights_history_2d[:, 1].min(), weights_history_2d[:, 1].max()

# Adiciona margem de 50% em cada direção
w0_margin = (w0_max - w0_min) * 0.5
w1_margin = (w1_max - w1_min) * 0.5

w0_range = np.linspace(w0_min - w0_margin, w0_max + w0_margin, 100)
w1_range = np.linspace(w1_min - w1_margin, w1_max + w1_margin, 100)
W0, W1 = np.meshgrid(w0_range, w1_range)

# Calcula loss para cada combinação de pesos
Z = np.zeros_like(W0)

# Se temos 3 pesos (2 features + bias), precisamos fixar o bias
if num_weights_2d == 3:
    bias_value = w_2d[2, 0]  # Valor final do bias
    for i in range(W0.shape[0]):
        for j in range(W0.shape[1]):
            w_temp = np.array([[W0[i, j]], [W1[i, j]], [bias_value]])
            Z[i, j] = compute_loss_for_weights(w_temp, X_reduced, y)
else:
    for i in range(W0.shape[0]):
        for j in range(W0.shape[1]):
            w_temp = np.array([[W0[i, j]], [W1[i, j]]])
            Z[i, j] = compute_loss_for_weights(w_temp, X_reduced, y)

# Visualização 3D com superfície de loss
fig = plt.figure(figsize=(18, 12))

# Vista 3D com superfície
ax1 = fig.add_subplot(221, projection='3d')
surface = ax1.plot_surface(W0, W1, Z, cmap='viridis', alpha=0.6, edgecolor='none', linewidth=0, antialiased=True)
iterations = np.arange(len(weights_history_2d))
scatter = ax1.scatter(weights_history_2d[:, 0], 
                      weights_history_2d[:, 1], 
                      loss_2d,
                      c=iterations, 
                      cmap='hot', 
                      s=3,
                      alpha=0.8)
ax1.plot(weights_history_2d[:, 0], 
         weights_history_2d[:, 1], 
         loss_2d, 
         'r-', 
         alpha=0.5, 
         linewidth=1.5,
         label='Trajetoria')
ax1.scatter([weights_history_2d[0, 0]], 
            [weights_history_2d[0, 1]], 
            [loss_2d[0]], 
            c='lime', 
            s=200, 
            marker='o', 
            label='Inicio',
            edgecolors='black',
            linewidths=2)
ax1.scatter([weights_history_2d[-1, 0]], 
            [weights_history_2d[-1, 1]], 
            [loss_2d[-1]], 
            c='red', 
            s=200, 
            marker='*', 
            label='Final',
            edgecolors='black',
            linewidths=2)
ax1.set_xlabel(f'w_{top_2_indices[0]}', fontsize=12, fontweight='bold')
ax1.set_ylabel(f'w_{top_2_indices[1]}', fontsize=12, fontweight='bold')
ax1.set_zlabel('Loss', fontsize=12, fontweight='bold')
ax1.set_title('Trajetoria 3D da Descida sobre Superficie de Loss', fontsize=14, fontweight='bold')
ax1.legend()
ax1.view_init(elev=25, azim=45)

# Projeção 2D com curvas de nível
ax2 = fig.add_subplot(222)
levels = 30
contour = ax2.contour(W0, W1, Z, levels=levels, cmap='viridis', alpha=0.6, linewidths=0.5)
ax2.clabel(contour, inline=True, fontsize=7, fmt='%.4f')
contourf = ax2.contourf(W0, W1, Z, levels=levels, cmap='viridis', alpha=0.4)

# Trajetória
trajectory = ax2.plot(weights_history_2d[:, 0], 
                      weights_history_2d[:, 1], 
                      'r-', 
                      alpha=0.7, 
                      linewidth=2,
                      label='Trajetoria')
# Mostra pontos a cada 50 iterações para não poluir
step = max(1, len(weights_history_2d) // 50)
scatter2 = ax2.scatter(weights_history_2d[::step, 0], 
                       weights_history_2d[::step, 1], 
                       c=iterations[::step], 
                       cmap='hot', 
                       s=30, 
                       alpha=0.8,
                       edgecolors='black',
                       linewidths=0.5,
                       zorder=5)
ax2.scatter([weights_history_2d[0, 0]], 
            [weights_history_2d[0, 1]], 
            c='lime', 
            s=200, 
            marker='o', 
            label='Inicio', 
            edgecolors='black', 
            linewidths=2,
            zorder=10)
ax2.scatter([weights_history_2d[-1, 0]], 
            [weights_history_2d[-1, 1]], 
            c='red', 
            s=200, 
            marker='*', 
            label='Final', 
            edgecolors='black', 
            linewidths=2,
            zorder=10)
ax2.set_xlabel(f'w_{top_2_indices[0]}', fontsize=12, fontweight='bold')
ax2.set_ylabel(f'w_{top_2_indices[1]}', fontsize=12, fontweight='bold')
ax2.set_title('Trajetoria 2D sobre Curvas de Nivel de Loss', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)
cbar2 = plt.colorbar(contourf, ax=ax2, label='Loss')

plt.show()

# Gráfico de barras: Importância dos pesos
plt.figure(figsize=(14, 6))
x_pos = np.arange(num_features)
colors = ['red' if i in top_2_indices else 'steelblue' for i in range(num_features)]
bars = plt.bar(x_pos, importance_score, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
plt.xlabel('Índice do Peso', fontsize=14, fontweight='bold')
plt.ylabel('Score de Importância', fontsize=14, fontweight='bold')
plt.title('Importância dos Pesos (Score Combinado)', fontsize=18, fontweight='bold')
plt.xticks(x_pos, [f'w_{i}' for i in range(num_features)])
plt.grid(axis='y', alpha=0.3)

for i, (idx, score) in enumerate(zip(x_pos, importance_score)):
    if idx in top_2_indices:
        plt.text(i, score + 0.005, 'TOP 2', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()